In [1]:
!pip install ffmpeg-python

In [187]:
import torch
from torch.nn import functional as F
import ffmpeg
import PIL
import numpy as np
import torchvision
from tqdm.notebook import tqdm
import json

In [100]:
from IPython import display
from PIL import Image

In [4]:
from google.colab import drive
drive.mount('/content/drive')
PREFIX = "/content/drive/MyDrive/traffic-light-detector"

Mounted at /content/drive


In [191]:
OBJECT_DETECTION_MODEL = f'{PREFIX}/pretrained/yolov5s.pt'
CLASSIFICATION_MODEL = f'{PREFIX}/pretrained/signal_classifier.torch'
DEVICE = torch.device('cuda')
FRAME_SKIPPING = 2
VIDEO_PATH = "/data/phase_I/video_2.mp4"
VIDEO_WIDTH = 640

In [192]:
def classify(image, bboxes, coord_coef):
  def coord_transform(c):
    return int(c*coord_coef)
  frame_results = {}
  for tid, bbox in enumerate(bboxes):
      traffic_light = torch.unsqueeze(torch.tensor(image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2]), :], dtype=torch.float32).permute(2,0,1), 0)
      tl_classify = torchvision.transforms.functional.resize(traffic_light.to(DEVICE), size=(32,16))
      prediction = torch.argmax(tlclassifier(tl_classify)[0]).item()
      frame_results[str(tid)] = {
          "coords": list(map(coord_transform,bbox[:-2])),
          "state": ['red','yellow','green','unknown'][prediction],
          "affect": True
      }
  return frame_results

model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model=OBJECT_DETECTION_MODEL)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(630, 64)
        self.fc2 = torch.nn.Linear(64, 4)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, 1)
        return x

tlclassifier = torch.load(CLASSIFICATION_MODEL)

if torch.cuda.is_available:
  model.cuda()
  tlclassifier.cuda()

video_file = f"{PREFIX}{VIDEO_PATH}"
probe = ffmpeg.probe(video_file)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
old_width = int(video_info['width'])
old_height = int(video_info['height'])
num_frames = int(eval(video_info['duration'])*eval(video_info['avg_frame_rate']))

new_width = 640
new_height = int(old_height / (old_width / new_width))

coord_coef = old_width / new_width

frame_size = new_width*new_height*3
process = (
    ffmpeg
    .input(video_file)
    .filter('scale', new_width, new_height)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run_async(pipe_stdout=True)
)

results = {}

for j in range(num_frames):
  if j%2==0:
    frame = (
      np
      .frombuffer(process.stdout.read(frame_size), np.uint8)
      .reshape((new_height, new_width, 3))
    )
    prediction = model([frame]).xyxy[0]
    predicted_traffic_lights = prediction[prediction[:,5] == 9].cpu().numpy()
    if len(predicted_traffic_lights) > 0:
      results[str(j)] = classify(frame, predicted_traffic_lights, coord_coef)
      results[str(j+1)] = classify(frame, predicted_traffic_lights, coord_coef)

json.dump(results, open("results.json","w"))

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...



                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1    656896  


requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Model Summary: 283 layers, 7276605 parameters, 7276605 gradients

YOLOv5 🚀 2021-4-1 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)



Adding autoShape... 
